In [12]:
import datetime as dt
import json as js
import urllib.request

url='https://api.openweathermap.org/data/2.5/onecall?lat=64.286670&lon=27.622125&units=metric&appid=e25afeccd6e22a2e996bed2809e43452'


resp=urllib.request.urlopen(url)
jsfile_Tornio=js.load(resp)

def unix_to_UTC_time(time_survey):
    return dt.datetime.utcfromtimestamp(time_survey).strftime('%Y-%m-%d %H:%M:%S')

date_weather_info=[]
for each_day in jsfile_Tornio['daily']:
    date_predicted=unix_to_UTC_time(each_day['dt'])
    temp_max=each_day['temp']['max']
    temp_min=each_day['temp']['min']
    temp_avg=each_day['temp']['day']
    rain_info=each_day.get('rain',0) # if it was no rain on the surveyed day, then return value 0

    date_weather_info.append({
        'date_predicted':date_predicted,
        'temp_max':temp_max,
        'temp_min':temp_min,
        'temp_avg':temp_avg,
        'rain_info (mm)': rain_info
    })